[texte du lien](https://)# [import video ]

In [1]:
import requests
import zipfile
import os

# URL of the GitHub repository's zip file
repo_url = "https://github.com/fedei10/Car-Detection/archive/refs/heads/main.zip"
repo_path = "/content/Car-Detection"  # Directory where the repository will be extracted

# Download the zip file
response = requests.get(repo_url, stream=True)
zip_path = "/content/Car-Detection-main.zip"
if response.status_code == 200:
    with open(zip_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Repository downloaded successfully and saved to {zip_path}")
else:
    print(f"Failed to download the repository. Status code: {response.status_code}")

# Extract the zip file
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content")
    print(f"Repository extracted successfully to {repo_path}")

# Clean up: Remove the downloaded zip file
os.remove(zip_path)
print(f"Cleaned up the zip file: {zip_path}")






Repository downloaded successfully and saved to /content/Car-Detection-main.zip
Repository extracted successfully to /content/Car-Detection
Cleaned up the zip file: /content/Car-Detection-main.zip


In [2]:
matricule_model_path = "/content/Car-Detection-main/models/plate_detection.pt"  # Your plate detection model
safe_driving_model_path = "/content/Car-Detection-main/models/yolo_classification.pt"  # Your safe driving model

In [3]:
!pip install ultralytics
!pip install opencv-python
!pip install easyocr
!pip install Pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.0/899.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 18.1 MB/s eta 0:00:00


In [4]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import requests
import re
import easyocr
from PIL import Image, ImageDraw, ImageFont

import os

import numpy as np

from ultralytics.utils.plotting import Annotator, colors


# Log detected classes
def log_detected_classes(results):
    for result in results:
        for box in result.boxes:
            print(f"Detected class: {int(box.cls)}, Confidence: {box.conf[0]:.2f}")

# Extract text from the plate using EasyOCR
def read_text_with_easyocr(roi):
    result = reader.readtext(roi, detail=0)
    r = ' '.join(result)
    print(f"unclean text: {r}")
    r = re.sub(r'[\u0660-\u0669]', '', r)
    cleaned_text = re.findall(r'\d+', r)
    plate = ""
    print(f"cleaned text {cleaned_text}")
    if len(cleaned_text) == 2:
        left_numbers = cleaned_text[1]
        right_numbers = cleaned_text[0]
        plate = f"{right_numbers} تونس {left_numbers}"
    elif len(cleaned_text) == 1:
        left_numbers = cleaned_text[0]
        plate = f"ن.ت"+left_numbers
    return plate
#Annotation With Arabic Font using Pillow
def annotate_with_arabic_text(image, text, position, font_path="/content/Car-Detection-main/Matricule_Model/Traditional Arabic Regular.ttf", font_size=40, color=(0, 255, 0)):
    """
    Annotates the image with Arabic text using Pillow.
    """
    # Convert OpenCV image to a Pillow image
    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_image)

    # Load the font
    font = ImageFont.truetype(font_path, font_size)

    # Add text to the image
    draw.text(position, text, font=font, fill=color)

    # Convert the Pillow image back to OpenCV format
    annotated_image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    return annotated_image

# Calculate the percentage of black pixels in the plate region
def check_black_percentage(roi):
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)
    black_pixels = np.sum(thresh == 255)
    total_pixels = roi.shape[0] * roi.shape[1]
    black_percentage = (black_pixels / total_pixels) * 100
    return black_percentage


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
reader = easyocr.Reader(['en', 'ar'], gpu=True)


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

#  ***videos to frames***

In [ ]:
import cv2
from ultralytics import YOLO
import easyocr
import numpy as np
from google.colab.patches import cv2_imshow
import time


# Global variables for tracking vehicle IDs and positions
vehicle_speeds = {}  # Dictionary to store speed for each vehicle ID
previous_positions = {}  # Previous positions of vehicle IDs

# Constants for speed calculation
FPS = 30  # Frame rate of the video
PIXEL_TO_METER_RATIO = 0.05  # Conversion ratio: 1 pixel ≈ 0.05 meters (adjust based on your video)

# Define paths to your models
matricule_model_path = "Matricule_Yolo11m.pt"
safe_driving_model_path = "yolo_classification.pt"


def load_models():
    try:
        model = YOLO("yolo11x.pt")  # Vehicle detection model
        plate_model = YOLO(matricule_model_path)  # License plate detection model
        safe_driving_model = YOLO(safe_driving_model_path)  # Person classification model
        return model, plate_model, safe_driving_model
    except Exception as e:
        print(f"Error loading models: {e}")
        return None, None, None


def calculate_speed(vid_id, current_pos):
    if vid_id in previous_positions:
        prev_pos = previous_positions[vid_id]
        distance = np.linalg.norm(np.array(current_pos) - np.array(prev_pos))  # Euclidean distance
        speed = (distance * PIXEL_TO_METER_RATIO) * FPS * 3.6  # Convert to km/h
        vehicle_speeds[vid_id] = round(speed, 2)
    else:
        vehicle_speeds[vid_id] = 0.0  # Initial speed is zero
    previous_positions[vid_id] = current_pos
    return vehicle_speeds[vid_id]


def read_text_with_easyocr(plate_roi):
    reader = easyocr.Reader(['en'])
    result = reader.readtext(plate_roi)
    plate_text = " ".join([res[1] for res in result])
    return plate_text


def annotate_with_arabic_text(frame, text, position):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    color = (0, 255, 0)
    thickness = 2
    cv2.putText(frame, text, position, font, font_scale, color, thickness, cv2.LINE_AA)
    return frame


def process_frame(frame, model, plate_model, safe_driving_model):
    # Run YOLO tracking
    results = model.track(frame, persist=True, tracker="bytetrack.yaml")  # Use ByteTrack for ID tracking

    for result in results[0].boxes:
        vid_id = int(result.id) if result.id else None
        cls = int(result.cls)
        x1, y1, x2, y2 = map(int, result.xyxy[0])

        if cls == 2 and vid_id:  # Vehicle class
            # Calculate speed
            center_position = ((x1 + x2) // 2, (y1 + y2) // 2)
            speed = calculate_speed(vid_id, center_position)

            # Draw vehicle bounding box and speed
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f"ID: {vid_id} Speed: {speed} km/h", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            # Detect persons within the vehicle ROI
            person_results = [box for box in results[0].boxes if int(box.cls) == 1]  # Person class
            for person in person_results:
                px1, py1, px2, py2 = map(int, person.xyxy[0])
                if x1 <= px1 <= x2 and y1 <= py1 <= y2:  # Check if person is inside vehicle ROI
                    # Extract person ROI and validate
                    person_roi = frame[py1:py2, px1:px2]
                    if person_roi.size > 0:  # Ensure ROI is not empty
                        # Classify the driving state
                        driving_results = safe_driving_model(person_roi, verbose=False)
                        top_class_idx = int(driving_results[0].probs.top1)
                        confidence = driving_results[0].probs.top1conf
                        class_name = driving_results[0].names[top_class_idx]

                        # Annotate the classification result
                        label = f"{class_name} ({confidence:.2f})"
                        cv2.putText(frame, label, (px1, py1 - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

            # Detect license plates within vehicle ROI
            vehicle_roi = frame[y1:y2, x1:x2]
            if vehicle_roi.size > 0:  # Ensure ROI is not empty
                plate_results = plate_model(vehicle_roi)
                for plate in plate_results[0].boxes:
                    px1, py1, px2, py2 = map(int, plate.xyxy[0])
                    plate_roi = vehicle_roi[py1:py2, px1:px2]
                    if plate_roi.size > 0:  # Ensure ROI is not empty
                        plate_text = read_text_with_easyocr(plate_roi)
                        frame = annotate_with_arabic_text(frame, plate_text, (x1 + px1, y1 + py1 - 30))
                        cv2.rectangle(frame, (x1 + px1, y1 + py1), (x1 + px2, y1 + py2), (0, 255, 0), 2)

    return frame


def main():
    # Load models
    model, plate_model, safe_driving_model = load_models()
    if model is None or plate_model is None or safe_driving_model is None:
        return

    # Open the video file
    video_path = "/content/Car-Detection-main/testvideos/VID-20241216-WA0004.mp4"
    cap = cv2.VideoCapture(video_path)

    # Frame skipping parameters
    skip_frames = 5  # Process every 5th frame
    frame_counter = 0

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break  # End of video

        frame_counter += 1
        if frame_counter % skip_frames != 0:
            continue  # Skip this frame

        # Process the frame
        frame = process_frame(frame, model, plate_model, safe_driving_model)

        # Display the frame
        cv2_imshow(frame)

        # Break if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
